# This is taken from Eric Vos's Rank 2 Solution for MachineHack Hackathon in April 2021
https://www.kaggle.com/a45632/rank-2-solution/execution
Many thanks to him for sharing his notebook and data!

In [ ]:
### Install autoviml here ######
#!pip install autoviml

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
from autoviml.Auto_ViML import Auto_ViML
import datetime

# For example, by running this (it will automatically load kaggle datasets in this directory)

from load_kaggle import load_kaggle


In [ ]:
sub, train, test = load_kaggle()
print(train.shape,test.shape)

In [ ]:
for i in sub.columns:
    if i != 'Uniq Id':
        del sub[i]
sub['Per Person Price'] = -1
del sub['Uniq Id']

In [ ]:
test['Per Person Price'] = 0
#append test to train for feature creation
full=train.append(test)
full=full.reset_index()
full=full.drop('index',axis=1)
print(full.shape)

In [ ]:
# various data correction & preprocessing
full['Hotel Details'] = full['Hotel Details'].str.replace(":One",':1.0')
full['Hotel Details'] = full['Hotel Details'].str.replace(":Two",':2.0')
full['Hotel Details'] = full['Hotel Details'].str.replace(":Three",':3.0')
full['Hotel Details'] = full['Hotel Details'].str.replace(":Four",':4.0')
full['Hotel Details'] = full['Hotel Details'].str.replace(":Five",':5.0')
full['Itinerary'] = ' ' + full['Itinerary'].astype(str)
full['Sightseeing Places Covered'] = full['Sightseeing Places Covered'].str.replace(")",'')
full['Sightseeing Places Covered'] = full['Sightseeing Places Covered'].str.replace("(",'')
full.iloc[27211,5] = full.iloc[27211,3]
full['Sightseeing Places Covered'] = full['Sightseeing Places Covered'].str.lower()
full['Hotel Details'] = full['Hotel Details'].str.lower()
full['Itinerary'] = full['Itinerary'].str.lower()
full['Airline'] = full['Airline'].str.lower()

In [ ]:
train.columns

# Feature Encoding

In [ ]:
# extract and create features from destination
alldest = []
temp = full['Destination']
for i in temp:
    truc = i.split('|')
    for i in truc:
        # Island 
        alldest.append(i)
dest = list(set(alldest))
clean = ['Singapore Cruise','Kovalam and Poovar']
for i in dest:
    if i not in clean:
        full[i] = False
        full.loc[full['Destination'].str.contains(i), i] = True 
len(dest)

In [ ]:
#sorted(dest)

In [ ]:
#full[full['Singapore']==True]

In [ ]:
# extract and create features from destination Hotel Details
full['mmt'] = False
full.loc[full['Hotel Details'].str.contains('mmt'), 'mmt'] = True

full['Hotel Details'] = full['Hotel Details'].str.replace(" - mmt holidays special",'')
full['Hotel Details'] = full['Hotel Details'].str.replace("- mmt holidays special",'')
full['Hotel Details'] = full['Hotel Details'].str.replace("-mmt holidays special",'')
full['Hotel Details'] = full['Hotel Details'].str.replace("(mmt holidays special)",'')
full['Hotel Details'] = full['Hotel Details'].str.replace(" - mmt special",'')
full['Hotel Details'] = full['Hotel Details'].str.replace("- mmt speciall",'')
full['Hotel Details'] = full['Hotel Details'].str.replace("-mmt special",'')
full['Hotel Details'] = full['Hotel Details'].str.replace("mmt special",'')

hotels = []
temp = full['Hotel Details']
for i in tqdm(temp):
    truc = i.split('|')
    for i in truc:
        keep = ''
        if i[-4] == ":":
            keep = i[:-4]
        else:
            if i[-3] == ":":
                keep = i[:-3]
            else:
                if i[-2] == ":":
                    keep = i[:-2]
                else:
                    if i[-1] == ":":
                        keep = i[:-1]
                    
        hotels.append(keep)
hotels = list(set(hotels))
for i in tqdm(hotels):
    full[i] = False
    full.loc[full['Hotel Details'].str.contains(i), i] = True 
len(hotels)

In [ ]:
# extract and create features from Sightseeing Places Covered
full['mmt2'] = False
full.loc[full['Sightseeing Places Covered'].str.contains('mmt'), 'mmt2'] = True
full['bus'] = False
full.loc[full['Sightseeing Places Covered'].str.contains('bus'), 'bus'] = True

full['Sightseeing Places Covered'] = full['Sightseeing Places Covered'].str.replace("mmt - ",'')
full['Sightseeing Places Covered'] = full['Sightseeing Places Covered'].str.replace(" - mmt ",'')

full['Sightseeing Places Covered'] = full['Sightseeing Places Covered'].str.replace("north",'')
full['Sightseeing Places Covered'] = full['Sightseeing Places Covered'].str.replace("south",'')

exc = []
temp = full['Sightseeing Places Covered']
for i in temp:
    truc = i.split('|')
    for i in truc:
        exc.append(i)
exc = list(set(exc))
for i in tqdm(exc):
    full[i] = False
    full.loc[full['Sightseeing Places Covered'].str.contains(i), i] = True 

len(exc)


# Feature Engineering

In [ ]:
def sumdigits(str):
    return sum(int(c) for c in str if c.isdigit())

def feature_engineering(df):
    #'Package Type'
    df['Package Type'].replace('Budget', 0,inplace=True)
    df['Package Type'].replace('Standard', 1,inplace=True)
    df['Package Type'].replace('Deluxe', 2,inplace=True)
    df['Package Type'].replace('Luxury', 3,inplace=True)
    df['Package Type'].replace('Premium', 4,inplace=True)
    #'Itinerary'
    duration = []
    for i in range(0,len(df)):
        duration.append(sumdigits(df.iloc[i,4]))
    df['duration'] = duration
    #'Places Covered'
    places = []
    temp = df['Places Covered']
    for i in temp:
        truc = i.split('|')
        places.append(len(truc))
    df['places'] = places
    #'Travel Date'
    #full['Travel Date'] = pd.to_datetime(full['Travel Date'], format='%d-%m-%Y')
    #add_datepart(full,'Travel Date',prefix='day$',drop=False,time=False)
    #df['day'] = df['Travel Date'].str[0:2]
    #df['month'] = df['Travel Date'].str[3:5]
    #df['year'] = df['Travel Date'].str[-4:]
    #'Hotel Details'
    score =[]
    temp = df['Hotel Details']
    for i in tqdm(temp):
        stars = 0.
        if i == 'Not Available':
            score.append(stars)
        else:
            truc = i.split('|')
            tot = 0
            for j in range(0,len(truc)):# pattern :x.x
                if truc[j][-4:-3]==':':
                    stars += float(truc[j][-3:])
                    tot+=1
                else:# pattern :x.x
                    if truc[j][-2:-1]==':':
                        stars += float(truc[j][-1:])
                        tot+=1
            if stars != 0.:
                stars = stars / tot
            score.append(stars)
    df['score'] = score
    df['score'] = df['score'].replace(0., np.nan)
    #'Start City'
    df['Start City'].replace('New Delhi', 0,inplace=True)
    df['Start City'].replace('Mumbai', 1,inplace=True)
    #'Airline'
    airlines = []
    temp = df['Airline']
    for i in temp:
        truc = i.split('|')
        airlines.append(len(truc))
    df['Airnum'] = airlines
    #'Flight Stops'
    #'Meals'
    #'Sightseeing Places Covered'
    visits = []
    temp = df['Sightseeing Places Covered']
    for i in temp:
        truc = i.split('|')
        visits.append(len(truc))
    df['visits'] = visits
    #'Cancellation Rules'
    temp = df['Cancellation Rules'].unique()
    for i in range(0,len(temp)):
        df['Cancellation Rules'].replace(temp[i], i,inplace=True)
    #'Per Person Price'
    
    ##################################################################
    #'Uniq Id'
    del df['Uniq Id']
    #'Package Name'
    del df['Package Name']
    #'Destination'
    del df['Destination']
    #'Itinerary'
    del df['Itinerary']
    #'Places Covered'
    del df['Places Covered']
    #'Travel Date'
    del df['Travel Date'] 
    #'Hotel Details'
    del df['Hotel Details']
    #'Airline'
    #del df['Airline']
    #'Sightseeing Places Covered'
    del df['Sightseeing Places Covered']


In [ ]:
feature_engineering(full)

In [ ]:
train = full[full['Per Person Price']!=0]
test = full[full['Per Person Price']==0]

In [ ]:
train['Per Person Price'] = np.log(train['Per Person Price'])

# Model 1 : Auto ViML

In [ ]:
target  = 'Per Person Price'
model, features, trainm, testm = Auto_ViML(
    train,
    target,
    test,
    sub,
    hyper_param="RS",
    feature_reduction=False,
    scoring_parameter= "neg_mean_squared_log_error",#
    KMeans_Featurizer=True,#False,
    Boosting_Flag= 'catboost',#True,#
    Binning_Flag=False,
    Add_Poly=1,
    Stacking_Flag=False,
    Imbalanced_Flag=False,
    verbose=2
)

In [ ]:
sub['Per Person Price'] = np.expm1(sub['Per Person Price'])
sub.head()

In [ ]:
#sub.to_csv('baseline_v20kc.csv', index=False)# catboost RMSLE = 0.1659